In [ ]:
!git clone https://github.com/AdrianoMedeirosPinheiro/CNN.git

In [ ]:
import zipfile

# Crie um novo diretório para extrair o arquivo zipado
!mkdir dataset
!mkdir DataSet

# Extraia o conteúdo do arquivo zipado no novo diretório
with zipfile.ZipFile('/content/CNN/leopardo.zip', 'r') as zip_ref:
    zip_ref.extractall('dataset')

with zipfile.ZipFile('/content/CNN/onca.zip', 'r') as zip_ref:
    zip_ref.extractall('dataset')


# Redimensionando as Imagens

In [ ]:
from PIL import Image
import os

# Dimensões da imagem redimensionada
width = 100
height = 100

# Diretório com as imagens originais
original_dir = '/content/dataset'

# Diretório para salvar as imagens redimensionadas
resized_dir = '/content/DataSet'

# Cria o diretório para salvar as imagens redimensionadas, se ele não existir
if not os.path.exists(resized_dir):
    os.makedirs(resized_dir)

# Percorre todas as imagens no diretório original
for file in os.listdir(original_dir):
      
# Abre a imagem original
  with Image.open(os.path.join(original_dir, file)) as im:
        
       
# Redimensiona a imagem
        im = im.resize((width, height))
  
# Salva a imagem redimensionada no diretório de destino
        im.save(os.path.join(resized_dir, file))

# DATASET

In [ ]:
import os
import numpy as np
from matplotlib import image


# Caminho para a pasta com as imagens
path = '/content/DataSet'

# Inicializa um array vazio para armazenar as imagens
images = []

# Percorre todos os arquivos na pasta
for file in os.listdir(path):
    # Lê a imagem
    img = image.imread(os.path.join(path, file))
    # Adiciona a imagem ao array
    images.append(img)

# Converte o array de imagens em um array NumPy
images = np.array(images)

In [ ]:
images.shape


In [ ]:
classe = []

for i in range(1, 41):
  classe.append(0)

for i in range(41, 81):
  classe.append(1)  

classe_array = np.asarray(classe)

In [ ]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(images, classe_array, train_size=0.8)

In [ ]:
y_train.shape

In [ ]:
#X_test.shape
X_train.shape

In [ ]:
X_train = X_train.reshape(64, 100, 100, 3)
X_test = X_test.reshape(16, 100, 100, 3)

In [ ]:
X_train.shape

In [ ]:
from keras.utils import to_categorical

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Rede Neural


## Construindo a Rede Neural Convolucional

### Definindo o modelo

In [ ]:
import tensorflow as tf

In [ ]:
model = tf.keras.models.Sequential()

### Adicionado a primeira camada de convolução

Hyper-parâmetros da camada de convolução:
- filters (filtros): 32
- kernel_size (tamanho do kernel): 3
- padding (preenchimento): valid
- função de ativação: relu
- input_shape (camada de entrada): (100, 100, 3)


In [ ]:
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding="valid", activation="relu", input_shape=(100,100,3)))

### Adicionando a segunda camada de convolução e a camada de max-pooling

Hyper-parâmetros da camada de convolução:
- filters (filtros): 32
- kernel_size (tamanho do kernel):3
- padding (preenchimento): valid
- função de ativação: relu

Hyper-parâmetros da camada de max-pooling:
- pool_size: 2
- strides: 2
- padding: valid

In [ ]:
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding="valid", activation="relu"))

In [ ]:
model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))

### Adicionando a terceira camada de convolução

Hyper-parâmetros da camada de convolução:

    filters: 64
    kernel_size:3
    padding: valid
    activation: relu


In [ ]:
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding="valid", activation="relu"))

###  Adicionando a quarta camada de convolução e a camada de max pooling

Hyper-parâmetros da camada de convolução:

    filters: 64
    kernel_size:3
    padding: valid
    activation: relu

Hyper-parâmetros da camada de max pooling:

    pool_size: 2
    strides: 2
    padding: valid


In [ ]:
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding="valid", activation="relu"))

In [ ]:
model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))

### Adicionando a camada de flattening

In [ ]:
model.add(tf.keras.layers.Flatten())

### Adicionando a primeira camada densa (fully-connected)

Hyper-parâmetros da camada densa:
- units/neurônios: 128
- função de ativação: relu

In [ ]:
model.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Adicionando a camada de saída

Hyper-parâmetros da camada de saída:

 - units/neurônios: 10 (número de classes)
 - activation: softmax


In [ ]:
model.add(tf.keras.layers.Dense(units=2, activation='softmax'))

In [ ]:
model.summary()

### Compilando o modelo



In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

### Treinando o modelo

In [ ]:
historico = model.fit(X_train, y_train, batch_size=64, epochs=25, validation_data=(X_test, y_test), verbose=1)

### Avaliando o modelo

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)

In [ ]:
print("Test accuracy: {}".format(test_accuracy))

In [ ]:
test_loss

In [ ]:
import matplotlib.pyplot as plt

acuracia_treino = historico.history['accuracy']
acuracia_teste = historico.history['val_accuracy']

epochs = range(1, len(acuracia_treino)+1) # x do gráfico

plt.plot(epochs, acuracia_treino, '-g', label='Acurácia Dados De Treino' )
plt.plot(epochs, acuracia_teste, '-b', label='Acurácia Dados De Teste' )
plt.legend()
plt.xlabel('epochs')
plt.ylabel('Acurácia')
plt.show()


# Predição

In [ ]:
import keras.utils as image
#from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from numpy import expand_dims, argmax

In [ ]:
from numpy import expand_dims, argmax
imgW, imgH = 100, 100

file = '/content/leopardo (1).jpg'

with Image.open(os.path.join(original_dir, file)) as im:

  im = im.resize((width, height))

y = model.predict(expand_dims(
image.img_to_array(im), axis = 0))

print('Animal in:', im, ':', classes[argmax(y)])

# Otimização

In [ ]:

from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten

# cria o modelo da rede neural
def create_model(num_filters, num_filters_2, kernel_size, pool_size, hidden_size):
  model = Sequential()
  model.add(Conv2D(num_filters, kernel_size, padding='valid', activation="relu", input_shape=(100,100,3)))
  model.add(Conv2D(num_filters, kernel_size, padding='valid', activation="relu"))
  model.add(MaxPooling2D(pool_size, strides=2, padding='valid'))
  model.add(Conv2D(num_filters_2, kernel_size, padding='valid', activation="relu"))
  model.add(Conv2D(num_filters_2, kernel_size, padding='valid', activation="relu"))
  model.add(MaxPooling2D(pool_size, strides=2, padding='valid'))
  model.add(Flatten())
  model.add(Dense(hidden_size, activation='relu'))
  model.add(Dense(2, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

# cria o modelo embutido na classe KerasClassifier
model = KerasClassifier(build_fn=create_model, verbose=0)

# define os parâmetros que serão otimizados
param_grid = {
    'num_filters': [32, 64, 128],
    'num_filters_2': [32, 64, 128],
    'kernel_size': [3, 5, 7],
    'pool_size': [2, 4, 6],
    'hidden_size': [128, 200, 250]
}

# cria o objeto de pesquisa em grade
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)

# ajusta o modelo usando os dados de treinamento e os parâmetros otimizados
grid_result = grid.fit(X_train, y_train)

# mostra os melhores parâmetros encontrados durante a otimização
print("Melhores parâmetros: {}".format(grid_result.best_params_))

# mostra a acurácia média para os melhores parâmetros
print("Acurácia média: {:.2f}%".format(grid_result.best_score_ * 100))


# Resnet

In [ ]:
# Primeiro, precisamos instalar as bibliotecas necessárias
!pip install torchvision

# Em seguida, importamos as bibliotecas que vamos precisar
import torch
import torchvision
import matplotlib.pyplot as plt

# Carregamos o modelo ResNet pré-treinado
model = torchvision.models.resnet18(pretrained=True)

# Desativamos os gradientes no modelo pré-treinado
for param in model.parameters():
  param.requires_grad = False

# Adicionamos uma camada linear no final do modelo pré-treinado
# para adaptá-lo às nossas classes personalizadas
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, len(classe_array))

# Definimos os parâmetros de treinamento
learning_rate = 0.001
num_epochs = 10

# Definimos o otimizador e a função de perda
optimizer = torch.optim.Adam(model.fc.parameters(), lr=learning_rate)
loss_fn = torch.nn.CrossEntropyLoss()

custom_train_dataloader = X_train

# Iteramos sobre os epochs
for epoch in range(num_epochs):
  # Iteramos sobre as imagens do conjunto de treinamento
  for images, labels in custom_train_dataloader:
    # Colocamos as imagens e rótulos no dispositivo apropriado (GPU ou CPU)
    images = images.to(device)
    labels = labels.to(device)

    # Zeramos os gradientes do otimizador
    optimizer.zero_grad()

    # Executamos o modelo e calculamos a perda
    outputs = model(images)
    loss = loss_fn(outputs, labels)

    # Realizamos o backpropagation e otimização
    loss.backward()
    optimizer.step()

  # Calculamos a precisão do conjunto de treinamento no final de cada epoch
  train_acc = calculate_accuracy(model, custom_train_dataloader, device)
  print(f"Epoch {epoch+1}/{num_epochs}: Training accuracy = {train_acc:.4f}")

# Calculamos a precisão do conjunto de teste no final do treinamento
test_acc = calculate_accuracy(model, custom_test_dataloader, device)
print(f"Test accuracy: {test_acc:.4f}")
